# Magnitude-intrinsic water-fat ambiguity can be resolved with multipeak fat modeling and a multipoint search method.

In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from TPTBox import to_nii,NII

sys.path.append("..")
sys.path.append("../../..")

from recon_mevibe import ti_ms_default,freqs_ppm,alpha_p
from mago_sp import magorino
## TODO Update file paths
# Load files

# Input
path = Path("/media/data/NAKO/dataset-nako/rawdata/100/100000/mevibe/")
s_0 = path / "sub-100000_sequ-53_acq-ax_part-eco0-opp1_mevibe.nii.gz"
s_1 = path / "sub-100000_sequ-54_acq-ax_part-eco1-pip1_mevibe.nii.gz"
s_2 = path / "sub-100000_sequ-55_acq-ax_part-eco2-opp2_mevibe.nii.gz"
s_3 = path / "sub-100000_sequ-56_acq-ax_part-eco3-in1_mevibe.nii.gz"
s_4 = path / "sub-100000_sequ-57_acq-ax_part-eco4-pop1_mevibe.nii.gz"
s_5 = path / "sub-100000_sequ-58_acq-ax_part-eco5-arb1_mevibe.nii.gz"
# Target output
use_target = True
water_file = path / "sub-100000_sequ-59_acq-ax_part-water_mevibe.nii.gz"
fat_file = path / "sub-100000_sequ-63_acq-ax_part-fat_mevibe.nii.gz"
r2s_file = path / "sub-100000_sequ-61_acq-ax_part-r2s_mevibe.nii.gz" # Note in our example we have to multiply with 0.1 to get the actual value




In [2]:

#path = Path("/media/data/NAKO/dataset-nako/rawdata/101/101000/mevibe/")
## Input
#s_0 = path / "sub-101000_sequ-53_acq-ax_part-eco0-opp1_mevibe.nii.gz"
#s_1 = path / "sub-101000_sequ-54_acq-ax_part-eco1-pip1_mevibe.nii.gz"
#s_2 = path / "sub-101000_sequ-55_acq-ax_part-eco2-opp2_mevibe.nii.gz"
#s_3 = path / "sub-101000_sequ-56_acq-ax_part-eco3-in1_mevibe.nii.gz"
#s_4 = path / "sub-101000_sequ-57_acq-ax_part-eco4-pop1_mevibe.nii.gz"
#s_5 = path / "sub-101000_sequ-58_acq-ax_part-eco5-arb1_mevibe.nii.gz"
## Target output
#use_target = True
#water_file = path / "sub-101000_sequ-59_acq-ax_part-water_mevibe.nii.gz"
#fat_file = path / "sub-101000_sequ-60_acq-ax_part-fat_mevibe.nii.gz"
#r2s_file = path / "sub-101000_sequ-62_acq-ax_part-r2s_mevibe.nii.gz" # Note in our example we have to multiply with 0.1 to get the actual value

MRI parameter:

- MagneticFieldStrength of the MRI device
- time_inversion in ms in the the same order as the $s_i$. See Dicom-Header.

In [ ]:
ti_ms = ti_ms_default
MagneticFieldStrength=3.0
print(f"{ti_ms_default= } \n{MagneticFieldStrength= }")
print('Fat model')
print(f"{alpha_p}")
print(f"{freqs_ppm}")

In [4]:

# NOTE we select slice 30 to make the example faster.
orientation = ("I","P","L")
s_idx = -25
magnitude_nii:list[NII] = [
    to_nii(s_0).reorient(orientation)[s_idx:s_idx+1,:,:],
    to_nii(s_1).reorient(orientation)[s_idx:s_idx+1,:,:],
    to_nii(s_2).reorient(orientation)[s_idx:s_idx+1,:,:],
    to_nii(s_3).reorient(orientation)[s_idx:s_idx+1,:,:],
    to_nii(s_4).reorient(orientation)[s_idx:s_idx+1,:,:],
    to_nii(s_5).reorient(orientation)[s_idx:s_idx+1,:,:],
]
magnitude = [a.get_array()[0] for a in magnitude_nii]
if use_target:
    water = to_nii(water_file).reorient(orientation).get_array()[s_idx]
    fat = to_nii(fat_file).reorient(orientation).get_array()[s_idx]
    r2s = to_nii(r2s_file).reorient(orientation).get_array()[s_idx] * 0.1

In [ ]:
def save_jpg(data,name="output",ending="jpg"):
    # Normalize to 0–255 for image representation
    normalized_data = ((data - np.min(data)) / (np.max(data) - np.min(data)) * 255).astype(np.uint8)

    # Convert to a PIL Image
    image = Image.fromarray(normalized_data)

    # Save as JPG
    image.save(f"{name}.{ending}")

    # Display in Jupyter notebook
    plt.imshow(normalized_data, cmap='gray')
    plt.axis('off')
    plt.title(name.replace("_"," "))
    plt.show()
if use_target:
    save_jpg(water, name="reference_water_map")

In [ ]:
from recon_mevibe import multipeak_fat_model_smooth,multipeak_fat_model_from_guess
from pipeline import predict_signal_prior
def count_percent(pred,water,fat, seg=None):
    if seg is None:
        seg = np.array(water+fat >= 50)
    total = seg.sum()
    dif_w = np.abs(pred-water.astype(float))
    dif_f = np.abs(pred-fat.astype(float))
    res = np.zeros_like(dif_f)
    res[dif_w<dif_f] = 1
    res[seg != 1] = 0
    print(res.sum(),total,res.sum()/total)
    return (res.sum(),total,res.sum()/total, res)


### Magnitude-intrinsic water-fat ambiguity can be resolved with multipeak fat modeling and a multipoint search method.

Source: https://pmc.ncbi.nlm.nih.gov/articles/PMC6593794/

In [7]:
#out_w, out_f, out_r, out_l = multipeak_fat_model_smooth(magnitude,smooth=False,rician_loss=False)
#save_jpg(out_w, name="MAGO")
#count_percent(out_w,water,fat);

In [8]:

#out_w, out_f, out_r, out_l = multipeak_fat_model_smooth(magnitude,smooth=True,rician_loss=False)
#save_jpg(out_w, name="MAGO residual-smoothed")

In [9]:
#out_w, out_f, out_r, out_l = multipeak_fat_model_smooth(magnitude,smooth=False,rician_loss=True)
#save_jpg(out_w, name="MAGORINO")
#count_percent(out_w,water,fat);


In [ ]:
out_w, out_f, out_r, out_l = magorino(magnitude,smooth=False,sigma=16)
save_jpg(out_w, name="MAGORINO")
count_percent(out_w,water,fat);


In [ ]:

out_w, out_f, out_r, out_l = multipeak_fat_model_smooth(magnitude,smooth=True,rician_loss=True)
save_jpg(out_w, name="MAGORINO residual-smoothed")

In [ ]:

pre = predict_signal_prior(magnitude_nii,"prior.nii.gz",gpu=0,ddevice="cuda")
save_jpg(to_nii(pre,False).get_array()[0], name="image2image prior")

In [ ]:

out_w, out_f, out_r, out_l = multipeak_fat_model_from_guess(magnitude,water,fat,MagneticFieldStrength=MagneticFieldStrength,ti_ms=ti_ms)
save_jpg(out_w, name="ours_MAGORINO")
count_percent(out_w,water,fat)

In [ ]:
out_w, out_f, out_r, out_l = multipeak_fat_model_from_guess(magnitude,water,fat,MagneticFieldStrength=MagneticFieldStrength,ti_ms=ti_ms,rician_loss=False)
save_jpg(out_w, name="ours_MAGO")
count_percent(out_w,water,fat)

In [ ]:
MAGORINO = "/media/data/robert/code/image2image/papers/vibe_inversion/tests/MAGORINO/100000/water.nii.gz"
MAGORINO =   to_nii(MAGORINO).reorient(orientation)[s_idx:s_idx+1,:,:]
MAGORINO = MAGORINO.get_array()[0]
MAGORINO[MAGORINO>=772] = 772
save_jpg(MAGORINO, name="MAGORINO")